# Connect the entire loop

## Load Data Generator and PDPTW Environment

In [85]:
#cd rl4co\ git

/Users/jiangwolin/Desktop/Research/llm-rl/rl4co git


In [108]:
# cwd => '/Users/jiangwolin/Desktop/Research/llm-rl/rl4co git'
%load_ext autoreload
%autoreload 2


import torch
from inner_loop.rl4co.models.zoo import AttentionModelPolicy, AttentionModel
from inner_loop.rl4co.envs.routing import SFGenerator, PDPTWEnv, PDPTWModEnv
import os

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [89]:
device = 'cpu'
env = PDPTWModEnv(generator_params={"num_customers":30}).reset(batch_size=[2]).to(device)
env # Contains user_id and flexibility

TensorDict(
    fields={
        action_mask: Tensor(shape=torch.Size([2, 61]), device=cpu, dtype=torch.bool, is_shared=False),
        capacity: Tensor(shape=torch.Size([2]), device=cpu, dtype=torch.float32, is_shared=False),
        current_node: Tensor(shape=torch.Size([2, 1]), device=cpu, dtype=torch.int64, is_shared=False),
        current_time: Tensor(shape=torch.Size([2, 1]), device=cpu, dtype=torch.float32, is_shared=False),
        demand: Tensor(shape=torch.Size([2, 61]), device=cpu, dtype=torch.float32, is_shared=False),
        done: Tensor(shape=torch.Size([2, 1]), device=cpu, dtype=torch.bool, is_shared=False),
        flexibility: Tensor(shape=torch.Size([2, 61]), device=cpu, dtype=torch.float32, is_shared=False),
        h3_indices: Tensor(shape=torch.Size([2, 61]), device=cpu, dtype=torch.int64, is_shared=False),
        i: Tensor(shape=torch.Size([2, 1]), device=cpu, dtype=torch.int64, is_shared=False),
        locs: Tensor(shape=torch.Size([2, 61, 2]), device=cpu, dt

## Load Ckpt to substitute oracle solver

In [95]:
env = PDPTWModEnv()
ckpt_path = os.getcwd() + "/inner_loop/examples/checkpoints/sf_newenv_2/epoch_epoch=067.ckpt"

torch.serialization.add_safe_globals([PDPTWModEnv])
ckpt = torch.load(ckpt_path, map_location="cpu", weights_only=False)
state = ckpt["state_dict"]

model = AttentionModel(env=PDPTWEnv())
model.load_state_dict(state, strict=False)

/opt/anaconda3/lib/python3.12/site-packages/lightning/pytorch/utilities/parsing.py:210: Attribute 'env' is an instance of `nn.Module` and is already saved during checkpointing. It is recommended to ignore them using `self.save_hyperparameters(ignore=['env'])`.
/opt/anaconda3/lib/python3.12/site-packages/lightning/pytorch/utilities/parsing.py:210: Attribute 'policy' is an instance of `nn.Module` and is already saved during checkpointing. It is recommended to ignore them using `self.save_hyperparameters(ignore=['policy'])`.


_IncompatibleKeys(missing_keys=[], unexpected_keys=['baseline.baseline.policy.encoder.init_embedding.project.weight', 'baseline.baseline.policy.encoder.init_embedding.project.bias', 'baseline.baseline.policy.encoder.net.layers.0.0.module.Wqkv.weight', 'baseline.baseline.policy.encoder.net.layers.0.0.module.Wqkv.bias', 'baseline.baseline.policy.encoder.net.layers.0.0.module.out_proj.weight', 'baseline.baseline.policy.encoder.net.layers.0.0.module.out_proj.bias', 'baseline.baseline.policy.encoder.net.layers.0.1.normalizer.weight', 'baseline.baseline.policy.encoder.net.layers.0.1.normalizer.bias', 'baseline.baseline.policy.encoder.net.layers.0.1.normalizer.running_mean', 'baseline.baseline.policy.encoder.net.layers.0.1.normalizer.running_var', 'baseline.baseline.policy.encoder.net.layers.0.1.normalizer.num_batches_tracked', 'baseline.baseline.policy.encoder.net.layers.0.2.module.lins.0.weight', 'baseline.baseline.policy.encoder.net.layers.0.2.module.lins.0.bias', 'baseline.baseline.policy

In [99]:
device = 'cpu'
td_init = PDPTWModEnv(generator_params={"num_customers":30}).reset(batch_size=[1]).to(device)
trained_policy = model.policy.to(device)
out = trained_policy(td_init.clone(), phase='test', decode_type="sampling", return_actions=True)
out

{'reward': tensor([-689.9833]),
 'log_likelihood': tensor([-31.8689], grad_fn=<SumBackward1>),
 'actions': tensor([[49, 19, 13, 20, 47, 59, 35, 15, 25, 48,  1,  2,  0, 11, 43, 12, 16, 60,
           0, 44, 14, 36, 53, 55, 26, 54, 56,  0, 50, 27, 37, 38, 28, 41,  3, 45,
          42, 57,  5, 21, 46,  7,  6, 22,  8,  9, 10,  0, 39, 40,  4, 33, 29, 34,
          31, 23, 32, 30, 17, 18,  0, 58, 51, 24, 52,  0]])}

## Connect Data Generator to PPO Environment

In [112]:
# Testing random policy
env = PDPTWModEnv()

# Policy: neural network, in this case with encoder-decoder architecture
policy = AttentionModelPolicy(env_name=env.name, 
                              embed_dim=128,
                              num_encoder_layers=3,
                              num_heads=8,
                            )
td = env.reset(batch_size=[1]) 
out = policy(td_init.clone(), phase="test", decode_type="greedy", return_actions=True)
env

PDPTWModEnv()